**Ch17-19: 决策树、误差及信息熵**

Source: 
[数据分析实战 45 讲](https://time.geekbang.org/column/intro/100021701?tab=intro)

---

# Decision Trees

## Information Gain for ID3 Algorithm

* Gain(D,a): D为父节点，a为此节点选择往下分采用的属性，一棵树往下面哪个分枝走取决于它**某一属性的取值有没有(True/False)超过那个阈值**，也就是决策树可视化后第一行<=___的那个数。

* 由信息增益的计算公式（父节点的信息熵减去所有子节点按照出现概率归一化后的信息熵）可知，信息增益指信息熵（没错就是它，代表不确定性）减少的程度，它越高（我们用增益最高的属性建树）信息熵下降的越多，**即此分枝下子节点的overall信息熵最小，纯度越高（即这一分类依据越果断，决策结果分歧越小，这个特征选择性越好）**。

* 然而该算法的一个问题：倾向于选择取值多的属性。

  * 如果把“编号”作为一个属性（把数据集分成特别特别多小子集，每个子集很显然只包含1个instance, 根据信息熵计算公式每个子节点信息熵全是0（1的log嘛毕竟，不像其他有更多instances的），再算信息增益肯定是Gain最大的情况了），那么“编号”会被选为最优属性。然而实际上“编号”是无关属性，对分类完全没有作用嘛。

  * **C4.5算法**改进了ID3算法，采用信息增益率，引入属性熵，避免了ID3的这个问题。除此之外，C4.5采用悲观剪枝（PEP）防止过拟合，并且可以离散化处理连续属性，选择具有最高信息增益的划分所对应的阈值。
 
* [作业题例子](https://time.geekbang.org/discuss/detail/86867)：有四个苹果{1+/2+/3-/4-}；两个属性：红{1+,2+}/不红{3-,4-} 和 大{1+,3-}/不大{2+,4-}；四个苹果两好两坏（这两个属性的父节点D）。

    * 根结点的信息熵：Ent(D)=1。
        
        - 以红属性划分 D1={1+,2+}, D2={3-,4-} Ent(D1)=0, Ent(D2)=0，Gain(D,红) = 1。同理方法计算以大划分Gain(D,大) = 0。因此选择红作为该节点最优特征。
        
        - 从这个节点往下走，在红为是作为父节点D的基础上，分析按大作为属性的信息增益。D = {2个好苹果}，Ent(D) = 0；D1(大 = 否)={1个好苹果}，D2(大 = 是)={1个好苹果} ；Ent(D1) = 0，Ent(D2) = 0；故Gain(D，大作为属性a) = 0
        
        - 因为大是与否在红决定的前提下对好苹果的决定没有影响，**所以剪去该分支**。 最终决策树：红苹果（是）-> 好苹果（是）; 红苹果（否）-> 好苹果（否）。
        
        - 在这一点上用CART决策树同理，都是如果有一个**子节点的不确定性是零了**，这个节点就可以作为**叶子节点**了，因为这一条线路再往下不确定性不可能被降低了。
     
-  CART决策树在分类任务中用基尼系数而不是信息增益。`DecisionTreeClassifier(criterion="gini")`是`criterion=`默认值，ID3的话就设成`"entropy"`。

* 有`max_depth`参数可以设置最大层数防止过拟合，也可以设置`max_leaf_nodes`。还有好多其他参数（反正都有default）比如`class_weight`: assigning weight to different classing labels.

## Regression Trees

* CART回归树划分数据集的过程和分类树的过程是一样的，只是回归树得到的预测结果是连续值（**已知分类类别求某一属性的可能取值**)

    * 回归树不用gini作为建树标准而是最小化样本离散程度（用到LAD最小绝对偏差和LSD最小二乘偏差）。

---

# 拿到数据之后如何处理！！

## Ch19: 利用Github中的泰坦尼克号数据（人员属性+存活与否）进行生存预测

* **数据探索**：`info()` to see dtypes and data integrity; `describe()` for statistics (only includes numeric columns); `describe(include=\['O'])` to view **'object'** (like strings) columns.

* **数据清洗、特征选择/feature extraction**：有缺失的要补齐，etc。对于特征选择：就像上述我们说ID3偏好取值多的属性（比如编号）一样，一定要把获取到的数据里面对分类用处不大的属性剔除掉，就比如这个编号。
    
    * 特征值里有些是string，应转化为数值类型，比如Sex = male/female变成0/1 -> sklearn's **DictVectorizer** class, 要先用`fit_transform()`。回忆Naive Bayes前用到的Tfidf*Vectorizer*那个。像这些需要把输入的特征值转化成一定的特征*向量*（或者特征矩阵/设计矩阵）的函数都除了一步拟合外还要有一步transform去转换，同理测试集也需要进行这一步转换！！！

- `fit_transform()`的'fit_'与真正训练模型(eg. 决策树还有朴素贝叶斯分类器）的`fit()`是不一样的！！！
前者是要转换数据但在转换前需要拟合训练集的mean和SD这些指标作为一个统一的标准，而后者就是为了训练模型优化损失函数什么的找到最优的模型函数参数！！

* **模型的训练、预测** (同前三块代码）

* **模型评估**。 没有同上，因为像此类数据集的测试集是没有提供groundtruth的！！！所以使用**cross_val_score**来评估，把原始的训练集平分为参数`cv`中设置的K份，每次将其中一份作为测试集，依次重复训练->测试->评估准确率。

In [9]:
# CART Classifying
from sklearn.model_selection import train_test_split # 把这些functions导进来，调用的时候直接就func名不用带前面的
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
import numpy as np

# dataset prep
iris = load_iris()
features = iris.data # training data
targets = iris.target # target: classification labels
print(features[:5],"\n",np.random.choice(targets, 5)) # 这俩都是array; features中有每一属性的值, target中0/1/2是iris分类的类别 (即species)

# splitting into train set and test set (by 33%)
tr_f, te_f, tr_t, te_t = train_test_split(features,targets,test_size=0.33,random_state=0)

# creating a CART classfier tree
classifier = DecisionTreeClassifier(criterion='gini') # this is the default 'criterion'
# **拟合**这个模型(即分类树)!!!!!
# most computationally expensive step in the entire code (where you TRAIN the model) 但是sklearn封装的太好了
classifier = classifier.fit(tr_f,tr_t)

# 封装得好吧，这一行就直接训练完了。接着predict（同样封装的好好，一行搞定）：
te_predicted = classifier.predict(te_f) 

prediction_accuracy = accuracy_score(te_t, te_predicted) # using the GROUNDTRUTH target
print(f"CART accuracy:{prediction_accuracy}")

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]] 
 [0 1 1 1 1]
CART accuracy:0.96


In [13]:
# CART Regression (数据集没了！！！只能看看代码）

# “分类树返回的是离散值”:你看上面嘛它返回的分类类别肯定是0/1/2；再看features数组里的值就是连续的了，毕竟是在某一属性上的取值嘛

# 本来这里应该用sklearn自带的波士顿房价数据集，该数据集给出了影响房价的一些指标，比如犯罪率等，最后给出了房价（一个属性），使用CART回归树对房价这一属性的值进行预测。

from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.datasets import load_boston
from sklearn.tree import DecisionTreeRegressor #不是classifier辽是regressor!

boston = load_boston()
print(boston.feature_names) # 数据探索阶段
features = boston.data
prices = boston.target # 这里的target不再是分类的label而是房价这个属性的值，我们要回归要先拟合它
print(features[:10],"\n",prices)

tr_f2, te_f2, tr_t2, te_t2 = train_test_split(features, prices, test_size=0.33)
regressor = DecisionTreeRegressor()
regressor.fit(tr_f2,tr_t2) 
te_predicted2 = regressor.predict(te_f2)
print('回归树二乘偏差均值:',mean_squared_error(te_t2, te_predicted2))
print('回归树绝对值偏差均值:',mean_absolute_error(te_t2, te_predicted2))

ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[:

In [21]:
# 作业题，用到DL里面那个老生常谈的手写数字集喵
from sklearn.datasets import load_digits
digits = load_digits()
digit_features = digits.data
digit_targets = digits.target

print("这是前三张image的feature_names：",digits.feature_names[:3],"\n是8x8像素的images input\n")
print("这是其features（像素点嘛）：\n",digit_features[:3])
print("这是targets：",digit_targets[:3])
tr_f3,te_f3,tr_t3,te_t3=train_test_split(digit_features,digit_targets,test_size=0.33)
clf=DecisionTreeClassifier(criterion='entropy') # 如果设='gini'（即默认参数）准确度算出来差不多
clf.fit(tr_f3,tr_t3)
te_p3=clf.predict(te_f3)
print("Accuracy score: ",accuracy_score(te_t3,te_p3),"\n\n真棒！！！")

这是前三张image的feature_names： ['pixel_0_0', 'pixel_0_1', 'pixel_0_2'] 
是8x8像素的images input

这是其features（像素点嘛）：
 [[ 0.  0.  5. 13.  9.  1.  0.  0.  0.  0. 13. 15. 10. 15.  5.  0.  0.  3.
  15.  2.  0. 11.  8.  0.  0.  4. 12.  0.  0.  8.  8.  0.  0.  5.  8.  0.
   0.  9.  8.  0.  0.  4. 11.  0.  1. 12.  7.  0.  0.  2. 14.  5. 10. 12.
   0.  0.  0.  0.  6. 13. 10.  0.  0.  0.]
 [ 0.  0.  0. 12. 13.  5.  0.  0.  0.  0.  0. 11. 16.  9.  0.  0.  0.  0.
   3. 15. 16.  6.  0.  0.  0.  7. 15. 16. 16.  2.  0.  0.  0.  0.  1. 16.
  16.  3.  0.  0.  0.  0.  1. 16. 16.  6.  0.  0.  0.  0.  1. 16. 16.  6.
   0.  0.  0.  0.  0. 11. 16. 10.  0.  0.]
 [ 0.  0.  0.  4. 15. 12.  0.  0.  0.  0.  3. 16. 15. 14.  0.  0.  0.  0.
   8. 13.  8. 16.  0.  0.  0.  0.  1.  6. 15. 11.  0.  0.  0.  1.  8. 13.
  15.  1.  0.  0.  0.  9. 16. 16.  5.  0.  0.  0.  0.  3. 13. 16. 16. 11.
   5.  0.  0.  0.  0.  3. 11. 16.  9.  0.]]
这是targets： [0 1 2]
Accuracy score:  0.8569023569023569 

真棒！！！


In [24]:
# L19: Titanic predictions

#数据获取、探索阶段
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction import DictVectorizer
train = pd.read_csv('L19_train_set.csv')
test = pd.read_csv('L19_test_set.csv')
print(train.info(),"\n")
print(train.describe(),"\n")
print(train.describe(include=['O']),"\n")
print(train.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None 

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486

In [26]:
# 数据清洗: spotted that 'Age', 'Cabin' and 'Embarked' all have NaN values
# Cabin has TOO MUCH NaNs so let's just drop it 喵喵喵喵。

train = train.drop(['Cabin'],axis=1)
test = test.drop(['Cabin'],axis=1)
train['Age'] = train['Age'].fillna(train['Age'].mean())
test['Age'] = test['Age'].fillna(test['Age'].mean())
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0]) 
# mode() can also be used for non-numeric cols, BUT!!!!!! it returns a Series ['S'] instead of the 'S' you want
test['Embarked'] = test['Embarked'].fillna(test['Embarked'].mode()[0])
print(train.info())

# 特征选择：spotted that 'PassengerId', 'Name', 'Ticket' has no use for a classifier. 
        # 'Survived' is the target value for classifying (the label) (刚才说了test set无groundtruth 即无此列！）
tr_features = train.drop(['PassengerId', 'Name', 'Ticket','Survived'],axis=1)
tr_targets = train['Survived']
te_features = test.drop(['PassengerId', 'Name', 'Ticket'],axis=1)
# 现在剩下的string类型列仅剩'Sex','Embarked'. 使用DictVectorizer()将符号表示的对象转化为数字0/1
dict_vect = DictVectorizer(sparse=False) # Sparse matrix: a matrix in which most of the elements are zero，用来节省空间和计算时间
tr_dict = tr_features.to_dict(orient='records') # orient: determines the type of the value in the dictionary. 'records': [{每行的column: dict value}, ..]
print(tr_dict[:5])
tr_features = dict_vect.fit_transform(tr_dict)
print(dict_vect.feature_names_,tr_features[:5],"\n注意看！自动把Embarked和Sex分别分成几个features用0/1二分")

# 模型训练、预测
model = DecisionTreeClassifier()
model.fit(tr_features,tr_targets)
te_predictions = model.predict(dict_vect.transform(te_features.to_dict(orient='records')))
print(te_predictions[:10])

# cross_val_score 交叉验证
print(f'\n交叉验证准确率为:{np.mean(cross_val_score(model,tr_features,tr_targets,cv=10)):.4f}, 棒！！') 

# 所以准确率验证这时候根本用不上我们萌萌的test prediction哇。

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 76.7+ KB
None
[{'Pclass': 3, 'Sex': 'male', 'Age': 22.0, 'SibSp': 1, 'Parch': 0, 'Fare': 7.25, 'Embarked': 'S'}, {'Pclass': 1, 'Sex': 'female', 'Age': 38.0, 'SibSp': 1, 'Parch': 0, 'Fare': 71.2833, 'Embarked': 'C'}, {'Pclass': 3, 'Sex': 'female', 'Age': 26.0, 'SibSp': 0, 'Parch': 0, 'Fare': 7.9

In [272]:
# 决策树可视化
import graphviz
from sklearn import tree
Titanic_tree = tree.export_graphviz(model,out_file=None)
Titanic_tree_plot = graphviz.Source(Titanic_tree)
Titanic_tree_plot.render("tree")
Titanic_tree_plot.view("Titanic_tree_plot")

'Titanic_tree_plot.pdf'